In [32]:
from PIL import Image
import requests
import torch
from transformers import BlipProcessor, BlipForQuestionAnswering, DefaultDataCollator, TrainingArguments, Trainer
from transformers import ViltProcessor, ViltForQuestionAnswering
import requests
from PIL import Image
import torch
import os
from os import listdir
from os.path import isfile, join
import json
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.utils.data import Dataset


In [ ]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="data/train.jsonl", split="train")

questions = [item for item in dataset["question"]]
images = [
        f"data/train_fill_in_blank/train_fill_in_blank/{pid}/image.png" for pid in dataset["pid"]
    ] 
answers = [item for item in dataset["answer"]]

# dataset = VQADataset(questions = questions,
#                           answers = answers,
#                           images = images,
#                           processor=processor)
batch_size = 16
# train_dataloader = DataLoader(dataset, collate_fn=collate_fn, batch_size=batch_size, shuffle=False)

In [ ]:
batch = next(iter(train_dataloader))
for k,v in batch.items():
    print(k, v.shape)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base", cache_dir="/home/congnguyen/drive/.cache/")
model = torch.load("./model/BLIP2/checkpoint_10")
model.to(device)
model.eval()
# prepare image + questio


In [33]:
eval_dir = os.path.abspath("data/test_data/test_data/")
eval_data = {}
for it in listdir(eval_dir):
    eval_data.update({it: []})
    for f in listdir(join(eval_dir, it)):
        file = join(eval_dir, it, f)
        eval_data[it].append(file)

def get_data(idx):
    img_url = eval_data[idx][0]
    text_url = eval_data[idx][1]
    textf = open(text_url)
    # returns JSON object as 
    # a dictionary
    data = json.load(textf)
    text = data["question"]
    if text == "What number is shown?":
        image = Image.open(img_url).convert("L")
    else:
        image = Image.open(img_url).convert("RGB")
    return image, text

In [ ]:
result = {}
for id in eval_data:
    image, text = get_data(id)
    encoding = processor(image, text, return_tensors="pt")
    encoding.to(device)
    # forward pass
    # outputs = model(**encoding)
    out = model.generate(**encoding, output_scores=True)
    # logits = outputs.logits
    # idx = logits.argmax(-1).item()
    print(out.scores)
    # out = processor.decode(out[0], skip_special_tokens=True).replace(" ","")
    # result.update({id: out})

In [ ]:
## result

In [ ]:
def write2csv(result, output='./results/blip5ep.csv'):
    import csv
    with open(output, 'w', newline='') as csvfile:
        fieldnames = ['ID', 'Label']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for idx in result:
            writer.writerow({'ID': idx, 'Label': result[idx]})

In [ ]:
## write2csv(result)

In [ ]:
import csv
def read_csv(filename):
    file = open(filename)
    csvreader = csv.reader(file)
    header = []
    header = next(csvreader)
    results = {}
    for row in csvreader:
        results.update({row[0]: row[1]}) 
    return results

In [ ]:
blip_result = read_csv("./results/blip5ep.csv")
bl20_result = read_csv("./results/bl20ep.csv")

In [ ]:
## blip_result

In [ ]:
count = 0
all = 0 
compares = {}
false = {}
for key in blip_result:
    all += 1
    if blip_result[key] == bl20_result[key]:
        compares.update({key: blip_result[key]})
        continue
    else:
        false.update({key: [blip_result[key], eval_data[key]]})
        count += 1

In [ ]:
# write2csv(compares, "./results/false_id.csv")

In [ ]:
# false

In [ ]:
import xlsxwriter


# Create an new Excel file and add a worksheet.
workbook = xlsxwriter.Workbook("./results/false_id.xlsx")
worksheet = workbook.add_worksheet()
# Widen the first column to make the text clearer.
worksheet.set_column("D:D", 40)
worksheet.set_column("E:E", 40)

count = 0
for key in false:
    # Insert an image.
    text_url = eval_data[key][1]
    textf = open(text_url)
    data = json.load(textf)
    text = data["question"]
    worksheet.set_row(count, 80)
    worksheet.write("A"+str(count), key)
    worksheet.write("B"+str(count), blip_result[key])
    worksheet.write("C"+str(count), bl20_result[key])
    worksheet.write("D"+str(count), text)
    worksheet.insert_image("E"+str(count), eval_data[key][0], {"x_scale": 0.5, "y_scale": 0.3})
    count += 1
    

workbook.close()

In [ ]:
## Training Data
def statistic(data):
    ques = {}
    tag = ["Move the ruler", "What is the area of the shape?", "How many shapes are", "How many", "What time", "Read the story.", "Fill in the blank to describe the model", "Use dice to measure", "Use paper clips", "What is the perimeter", "What number is shown?"]
    for q in data:
        flag = False
        for t in tag:
            if t in q:
                flag = True
                if t not in ques:
                    ques.update({t: 1})
                else:
                    ques[t] += 1
                break
        if flag == False:
            print(q)
            if q not in ques:
                ques.update({q: 1})
            else:
                ques[t] += 1
    return ques
    
ques = statistic(questions)
sum = 0
for q in ques:
    sum += ques[q]
print(sum)
print(len(questions))

In [ ]:
# ques

In [ ]:
unique_labels = list(set(answers))

label2id = {label: idx for idx, label in enumerate(unique_labels)}
id2label = {idx: label for label, idx in label2id.items()}

In [ ]:
# len(id2label)

In [ ]:
ques_test = []
for id in eval_data:
    text_url = eval_data[id][1]
    textf = open(text_url)
    data = json.load(textf)
    text = data["question"]
    ques_test.append(text)
ques = statistic(ques_test)

In [ ]:
sum = 0
for q in ques:
    sum += ques[q]
print(sum)
print(len(ques_test))

In [ ]:
false_test = []
for key in false:
    # Insert an image.
    text_url = eval_data[key][1]
    textf = open(text_url)
    data = json.load(textf)
    text = data["question"]
    false_test.append(text)

In [ ]:
ques = statistic(false_test)
sum = 0
for q in ques:
    sum += ques[q]
print(sum)
print(len(false_test))

In [ ]:
# ques

In [ ]:
import openpyxl
 
# Define variable to load the dataframe
dataframe = openpyxl.load_workbook("results/false_id.xlsx")
 
# Define variable to read sheet
dataframe1 = dataframe.active

true_id = {}
# Iterate the loop to read the cell values
for row in range(1, dataframe1.max_row):
    # print(dataframe1.cell(row = row, column = 1).value, ", ", 
    #       dataframe1.cell(row = row, column = 7).value, ", ",
    #       dataframe1.cell(row = row, column = 8).value)
    if dataframe1.cell(row = row, column = 7).value is not None and dataframe1.cell(row = row, column = 8).value is not None:
        value = dataframe1.cell(row = row, column = 8).value
        de = value-int(value)
        if de == 0:
            value = str(int(value))
            if len(value) == 4:
                value = value[0]+","+value[1::]
        else:
            value = str(value).replace(".", ",")
        true_id.update({dataframe1.cell(row = row, column = 1).value: value})


## 

In [ ]:
blip_result = read_csv("./results/blip5ep.csv")
bl20_result = read_csv("./results/bl20ep.csv")

In [ ]:
for k in true_id:
    if k == '96442':
        print(k, true_id[k])
    if k in blip_result:
        blip_result.update({k: true_id[k]})

In [ ]:
# eval_data

In [35]:
vilt_result = {}
def read_csv2(filename):
    file = open(filename)
    csvreader = csv.reader(file)
    header = []
    header = next(csvreader)
    vilt_result = {}
    for row in csvreader:
        vilt_result.update({row[0]: [row[1],row[2]]}) 
    return vilt_result
    
vilt_result = read_csv2("./results/vilt_ensemble.csv")

In [42]:
blipAug2 = read_csv("./results/blipAug31.csv")
blipGray2= read_csv("./results/blipGray2.csv")
vilt= read_csv("./results/vilt_ensemble.csv")
test = read_csv("./results/test.csv")

In [45]:
count = 0
true_ans = 0
for id in eval_data:
    image, text = get_data(id)
    if text == "What number is shown?":
        if test[id] == blipAug2[id]:
            true_ans += 1
            # print(id, ": True ", blipAug2[id])
        # else:
            # print(id, ": False ", blipAug2[id])
        count += 1

print(true_ans)
count

101


170

In [ ]:
## 99/170

In [37]:
fdup = 0
tdup = 0
tvilt = 0
tblipAug2 = 0
false_id = 0
for id in eval_data:
    image, text = get_data(id)
    if vilt[id] == blipAug2[id]:
        if vilt[id] != test[id]:
            fdup += 1
        else:
            tdup += 1
    elif vilt[id] != blipAug2[id]:
        if vilt[id] != test[id] and blipAug2[id] == test[id]:
            tblipAug2 += 1
            print("False: ", vilt_result[id][1])
        elif vilt[id] == test[id] and blipAug2[id] != test[id]:
            print("True: ", vilt_result[id][1])
            tvilt += 1
        else:
            false_id += 1
            print("AFalse: ", vilt_result[id][1])
            
print("Dup True: ", tdup)
print("Dup False: ", fdup)
print("Vilt True: ", tvilt)
print("Blip True: ", tblipAug2)
print("VilT&Blip False: ", false_id)
print(tdup+fdup+tvilt+tblipAug2+false_id)

True:  13.762677192687988
False:  -0.10565970838069916
AFalse:  -0.19564026594161987
AFalse:  -6.258315086364746
False:  5.291347503662109
AFalse:  6.595821857452393
AFalse:  3.9768478870391846
AFalse:  -0.9381645917892456
True:  14.527335166931152
AFalse:  1.5711500644683838
False:  0.24098752439022064
AFalse:  -0.34161120653152466
AFalse:  -2.648970127105713
False:  11.300838470458984
True:  13.517353057861328
AFalse:  -4.8656768798828125
AFalse:  3.1353724002838135
True:  14.55045223236084
True:  14.569816589355469
True:  12.903050422668457
True:  12.593175888061523
AFalse:  -0.013706590980291367
AFalse:  0.1866191327571869
True:  -0.4000682532787323
False:  -8.266329765319824
AFalse:  -0.16811597347259521
AFalse:  -6.460824489593506
True:  12.765212059020996
True:  11.127026557922363
False:  -2.8132405281066895
False:  10.419587135314941
True:  9.019761085510254
True:  13.378215789794922
AFalse:  -2.9545698165893555
False:  -1.0944095849990845
True:  15.039958000183105
AFalse:  -3.

## 